In [34]:
import heapq
from itertools import count


# Coordenadas iniciales y metas
iniciales = {'R': (2, 2), 'M1': (0, 0), 'M2': (2, 0), 'M3': (0, 3)}
metas = {'M1': (3, 3), 'M2': (3, 2), 'M3': (3, 1)}
barreras = {(1, 1), (0, 1)}  # posición del #

# Acciones posibles: (fila, columna)
acciones = [(-1,0), (1,0), (0,-1), (0,1)]

# Distancia de Manhattan
def manhattan(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])


In [35]:
class Estado:
    def __init__(self, robot, inventarios, cargando=None):
        self.robot = robot
        self.inventarios = inventarios  # {'M1': (x, y), ...}
        self.cargando = cargando  # None o 'M1', 'M2', 'M3'

    def es_objetivo(self):
        return all(self.inventarios[m] == metas[m] for m in metas)

    def __eq__(self, otro):
        return self.robot == otro.robot and self.inventarios == otro.inventarios and self.cargando == otro.cargando

    def __hash__(self):
        return hash((self.robot, frozenset(self.inventarios.items()), self.cargando))


In [36]:
def sucesores(estado):
    sucesores = []

    rx, ry = estado.robot

    # 1. Mover (lleva o no lleva estante)
    for dx, dy in acciones:
        nr, nc = rx + dx, ry + dy
        if 0 <= nr < 4 and 0 <= nc < 4 and (nr, nc) not in barreras:
            # Si lleva un estante, no puede pasar sobre otro estante
            if estado.cargando:
                # Verificamos si hay otro estante en la celda destino
                hay_estante = any((pos == (nr, nc) and m != estado.cargando)
                                for m, pos in estado.inventarios.items())
                if hay_estante:
                    continue  # No puede moverse ahí

            nuevas_pos = estado.inventarios.copy()
            if estado.cargando:
                nuevas_pos[estado.cargando] = (nr, nc)
            sucesores.append((
                Estado((nr, nc), nuevas_pos, estado.cargando),
                f"mover R fila{nr} columna{nc}"
            ))

    # 2. Cargar un estante (solo si no lleva otro)
    if estado.cargando is None:
        for m, (mx, my) in estado.inventarios.items():
            if (mx, my) == estado.robot:
                sucesores.append((
                    Estado(estado.robot, estado.inventarios.copy(), m),
                    f"cargar R {m} fila{rx} columna{ry}"
                ))

    # 3. Descargar (solo si lleva uno y está en su meta)
    if estado.cargando:
        meta_pos = metas[estado.cargando]
        if estado.robot == meta_pos:
            sucesores.append((
                Estado(estado.robot, estado.inventarios.copy(), None),
                f"descargar R {estado.cargando} fila{rx} columna{ry}"
            ))

    return sucesores


In [37]:

def heuristica(estado):
    total = 0
    for m in metas:
        if estado.cargando == m:
            # Si lo lleva, cuenta hasta su meta
            total += manhattan(estado.robot, metas[m])
        elif estado.inventarios[m] != metas[m]:
            # Si no lo lleva y no está en meta, cuenta ir + llevar
            total += manhattan(estado.robot, estado.inventarios[m]) + manhattan(estado.inventarios[m], metas[m])
    return total



def a_estrella_mostrar():
    inicial = Estado(
        iniciales['R'],
        {m: pos for m, pos in iniciales.items() if m != 'R'},
        cargando=None
    )
    frontera = []
    visitados = set()
    contador = count()

    heapq.heappush(frontera, (0, next(contador), 0, inicial, [], [inicial]))

    while frontera:
        _, _, g, actual, camino, historial = heapq.heappop(frontera)
        if actual in visitados:
            continue
        visitados.add(actual)

        if actual.es_objetivo():
            return camino, historial

        for sucesor, accion in sucesores(actual):
            if sucesor not in visitados:
                g_nuevo = g + 1
                h = heuristica(sucesor)
                f_nuevo = g_nuevo + h
                heapq.heappush(frontera, (
                    f_nuevo, next(contador), g_nuevo, sucesor, camino + [accion], historial + [sucesor]))

    return None, None


def imprimir_grilla(estado):
    grilla = [['.' for _ in range(4)] for _ in range(4)]

    # Poner paredes
    for (x, y) in barreras:
        grilla[x][y] = '#'

    # Poner inventarios
    for m, (x, y) in estado.inventarios.items():
        grilla[x][y] = m

    # Poner robot (puede estar encima de inventario, por eso va al final)
    rx, ry = estado.robot
    if grilla[rx][ry] in ['M1', 'M2', 'M3']:
        grilla[rx][ry] = f'R/{grilla[rx][ry]}'  # robot cargando
    else:
        grilla[rx][ry] = 'R'

    # Imprimir grilla
    for fila in grilla:
        print(' '.join(fila))
    print()



In [38]:
acciones, estados = a_estrella_mostrar()

if acciones:
    for i, (accion, estado) in enumerate(zip(acciones, estados[1:]), 1):
        print(f"\nPaso {i}: {accion}")
        imprimir_grilla(estado)

    print("\n🎯 Secuencia completa de acciones:")
    for paso in acciones:
        print(paso)
else:
    print("No se encontró una solución.")



Paso 1: mover R fila2 columna1
M1 # . M3
. # . .
M2 R . .
. . . .


Paso 2: mover R fila2 columna0
M1 # . M3
. # . .
R/M2 . . .
. . . .


Paso 3: cargar R M2 fila2 columna0
M1 # . M3
. # . .
R/M2 . . .
. . . .


Paso 4: mover R fila2 columna1
M1 # . M3
. # . .
. R/M2 . .
. . . .


Paso 5: mover R fila2 columna2
M1 # . M3
. # . .
. . R/M2 .
. . . .


Paso 6: mover R fila3 columna2
M1 # . M3
. # . .
. . . .
. . R/M2 .


Paso 7: descargar R M2 fila3 columna2
M1 # . M3
. # . .
. . . .
. . R/M2 .


Paso 8: mover R fila2 columna2
M1 # . M3
. # . .
. . R .
. . M2 .


Paso 9: mover R fila2 columna1
M1 # . M3
. # . .
. R . .
. . M2 .


Paso 10: mover R fila2 columna0
M1 # . M3
. # . .
R . . .
. . M2 .


Paso 11: mover R fila1 columna0
M1 # . M3
R # . .
. . . .
. . M2 .


Paso 12: mover R fila0 columna0
R/M1 # . M3
. # . .
. . . .
. . M2 .


Paso 13: cargar R M1 fila0 columna0
R/M1 # . M3
. # . .
. . . .
. . M2 .


Paso 14: mover R fila1 columna0
. # . M3
R/M1 # . .
. . . .
. . M2 .


Paso 15: 